In [ ]:
from __future__ import print_function
import os
import tensorflow as tf
import keras

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

from custom_model.layers_keras import *
from custom_model.model_keras import *
from custom_model.math_utils import *
from keras import metrics

import matplotlib.pyplot as plt
%matplotlib inline
import h5py
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [ ]:
dt = 2
OBS = 30//dt-1
PRED = 20//dt

In [ ]:
Data = h5py.File('Datasets/RRot_cc2_20.h5', 'r')
x_train = np.array(Data['Speed_obs_train'])
y_train = np.array(Data['Speed_pred_train'])
e_train = np.array(Data['E_train'])
x_test = np.array(Data['Speed_obs_test'])
y_test = np.array(Data['Speed_pred_test'])
e_test = np.array(Data['E_test'])

x_size = x_train.shape[1:]
y_size = y_train.shape[1:]

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('e_train shape:', e_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(y_train.shape[1:], 'output size')
'''
x_test = x_test[:3600]
y_test = y_test[:3600]
e_test = e_test[:3600]

x_test = np.delete(x_test, np.s_[3150:3180], axis=0)
y_test = np.delete(y_test, np.s_[3150:3180], axis=0)
e_test = np.delete(e_test, np.s_[3150:3180], axis=0)

x_test = x_test[:1440]
y_test = y_test[:1440]
e_test = e_test[:1440]

x_test = np.delete(x_test, np.s_[1260:1272], axis=0)
y_test = np.delete(y_test, np.s_[1260:1272], axis=0)
e_test = np.delete(e_test, np.s_[1260:1272], axis=0)
'''
x_train = x_train[:,-OBS:]
x_test = x_test[:,-OBS:]
y_train = y_train[:,:PRED]
e_train =e_train[:,:PRED]
y_test = y_test[:,:PRED]
e_test =e_test[:,:PRED]

print(x_train.shape)

In [ ]:
a,b,c = train_model(x_train, e_train, y_train, x_test, e_test, y_test, 
            OBS, PRED, decay=1e-2, k_max=5, rf=0, repeat=1, convergence=4, model_name='dfn',dgc_mode='dgc')# for studying

In [ ]:
print(a)
print(b)
print(c)

In [ ]:
model = create_embed_model(obs_timesteps=OBS, pred_timesteps=PRED, nb_nodes=dim, k=3, 
                   dgc_mode='dgc',model_name='dfn')

In [ ]:
model.summary()

In [ ]:
# Let's train the model
opt = keras.optimizers.Adam(lr=0.01, decay=1e-3)

model.compile(loss = 'mape',
              optimizer=opt,
              metrics=['mae',rmse])

callbacks = [EarlyStopping(monitor='val_loss', patience=2),
             #ModelCheckpoint(filepath='test.h5', save_weights_only=True, monitor='val_loss', save_best_only=True),
             ScheduledSampling(k=4)
            ]

history = model.fit([x_train,e_train], y_train,
          epochs=256,
          batch_size=256,
          callbacks=callbacks,
          #validation_data = ([x_val,e_val], y_val),
          shuffle='batch',
          validation_split = 0.2
                   )

In [ ]:
y = model.predict([x_test,e_test])

In [ ]:
for i in range(1,PRED+1):   
    print(MAP(y_test[:,:i]*120, y[:,:i]*120), ' ',100*MAPE(y_test[:,:i]*120, y[:,:i]*120), ' ',RMSE(y_test[:,:i]*120, y[:,:i]*120))
print('##########')
for i in range(PRED):
    print(MAP(y_test[:,i]*120, y[:,i]*120), ' ',100*MAPE(y_test[:,i]*120, y[:,i]*120), ' ',RMSE(y_test[:,i]*120, y[:,i]*120))

In [ ]:
from utils_vis import *
plotter(history)

In [ ]:
file=h5py.File('load_models/cc_dgfn_dt3.h5','r')
weight = []
for i in range(len(file.keys())):
    weight.append(file['weight'+str(i)][:])
model.set_weights(weight)

In [ ]:
file = h5py.File('load_models/cl_dgfn_dt3.h5','w')
weight = model.get_weights()
for i in range(len(weight)):
    file.create_dataset('weight'+str(i),data=weight[i])
file.close()

In [ ]:
day = 8
start = 150*(day-1)
end = start+150
j=2
gt = [y_test[i] for i in range(start,end,PRED)]
pred = [y[i][:j] for i in range(start,end,j)]

ground_truth = np.concatenate(gt, axis=0)
prediction = np.concatenate(pred, axis=0)

print(ground_truth.shape)
print(prediction.shape)

In [ ]:
plt.figure(figsize = (10,7))
plt.imshow(ground_truth.squeeze().transpose(),aspect = 'auto')
plt.title('matrix')
plt.colorbar()

In [ ]:
plt.figure(figsize = (10,7))
plt.imshow(prediction.squeeze().transpose(),aspect = 'auto')
plt.title('matrix')
plt.colorbar()